In [1]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image

from photutils.aperture import aperture_photometry, CircularAperture

# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

To do list
- [x] get centroid coordinates in a list for each image
- [ ] apply aperture photometry
- [ ] apply psf (gross, allegedly)

```python
from photutils.aperture import aperture_photometry, CircularAperture
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))  
apertures = CircularAperture(positions, r=4.0)  
phot_table = aperture_photometry(image, apertures)  
for col in phot_table.colnames:  
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table) 
```

In [2]:
def apply_aperture(images_dir, r=50.0):
    images = ImageFileCollection(images_dir)
    images = images.files_filtered(include_path=True)
    
    aperture_fluxes = []
    for i, path in np.ndenumerate(images):
        image = CCDData.read(path)
        position = np.transpose((
            image.header['X-CENTROID'],
            image.header['Y-CENTROID']
        ))
        aperture = CircularAperture(position, r=r)
        phot = aperture_photometry(image, aperture)[0]['aperture_sum']
        